In [ ]:
import requests
from bs4 import BeautifulSoup
from csv import writer
import pandas as pd
from collections import defaultdict

Record = pd.DataFrame()
A_Names = []
A_Links = []
P_Title = []
P_Links = []
Year = []

def extracting_data(no_of_pages):
    page_number = 0
    while page_number < no_of_pages:
        link_output= 'https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/?page='+str(page_number)
        link_response = requests.get(link_output)
        soup = BeautifulSoup(link_response.text, "html.parser")
        
        for link in soup.findAll('a', {'rel':['ContributionToBookAnthology',
                                              'ContributionToJournal',
                                              'ContributionToBookAnthology']}):
            
            publication_link = link.get('href')
            publication_title =link.select_one("span").text
            
            #queue containing urls
            P_Links.append(publication_link)
            P_Title.append(publication_title)
            
            #extracting publication url
            publication = requests.get(publication_link)
            P_soup = BeautifulSoup(publication.text)
            
            date = soup.find('span', class_='date')
            
            A_details = P_soup.find('a', {'class':'link person'}) or int(0)
            
            #conditions for author details 
            
            if A_details!=0:
                A_link = A_details.get('href')
                A_name = A_details.select_one("span").text
                
                A_Names.append(A_name)
                A_Links.append(A_link)
                
                Year.append(date.text)
            else:
                A_Names.append("0")
                A_Links.append("0")
                Year.append(date.text)
        page_number+=1   
    Record={"publication": P_Title,
            "publicationUrl": P_Links,
            "year": Year,
            "authorName": A_Names, 
            "authorUrl": A_Links}
    Dataframe = pd.DataFrame.from_dict(Record, orient='index')
    record_data_transpose = Dataframe.transpose()
    return record_data_transpose
                
Record = extracting_data(20)




#printing statements to get record according to publication title ,author_name and year  

Publication_Record =  Record['publication']
print('publication_title:\n',Publication_Record )

Publication_authorName =  Record['authorName']
print('authorName:\n',Publication_authorName)

Publication_year =  Record['year']
print('year:\n',Publication_year)




#importing Natural language Toolkit for stopwards in english 

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words_language= set(stopwords.words('english'))
print(stop_words_language)




#NLTK tokenization is to be done in this part

punctuation_data= '''["\\\')\\]}]+?(?:\\s+|(?=--)|$'''

nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

Stemmer= PorterStemmer()
preprocessed_document = []
for document in Publication_Record:
    tokens = word_tokenize(document)
    tmp = ""
    for word in tokens:
        if word not in stop_words_language and word not in punctuation_data:
            tmp += Stemmer.stem(word) + " "
    preprocessed_document.append(tmp)
     
print('Pre-Processed Document:\n\n',preprocessed_document)
Record['preprocessedTitle']=preprocessed_document
print('====================================================================\n=============================================')
print('Record:\n\n',Record)





# Indexer

from collections import defaultdict

D=[]
for eachWord in Record['preprocessedTitle'].str.split():
    D.append(eachWord)

index_=defaultdict(set)
for index, doc in enumerate(D):
    for word in doc:
        index_[word].add(index)
print('INDEXER:\n',index_)

def fetch_Record_result(searchText):
    keys = index_.keys()
    if searchText in keys:
        i = index_[searchText]
        for index_Key in i:
          return Record.loc[index_Key, :]
    


# query to run vertical search engine


Search_Engine = input("|||||WELCOME TO SEARCH ENGINE|||||:")
fetch_Record_result(Search_Engine)    